In [1]:
import jsonlines
import time
import datetime

def ContinuousInterval(intervalL):
    maxInt = []
    tempInt = [intervalL[0]]
    for q in range(1,len(intervalL)):
        if intervalL[q]-intervalL[q-1] > 1:
            if len(tempInt) > len(maxInt):
                maxInt = tempInt
            tempInt = [intervalL[q]]
        else:
            tempInt.append(intervalL[q])
    if len(maxInt)==0:
        maxInt = tempInt
    return maxInt


In [2]:
TIME_STEPS = 12

In [5]:
def preProcessData(filename):
    twitter = []
    with open(filename,encoding='utf-8') as f:
        for item in jsonlines.Reader(f):
            twitter.append(item)
    totalData = []
    for tweets in twitter:  #under one event
        TweetList = []
        for tweet in tweets:
            tweetText = tweet['text']
            tweetId = tweet['id']
            createTime = time.mktime(datetime.datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S +0000 %Y").timetuple())
            TweetList.append({'id':tweetId, 'text':tweetText, 'time': createTime})
        TweetList = sorted(TweetList, key=lambda k: k['time'])        #from early to late
        TotalTimeLine = TweetList[-1]['time']-TweetList[0]['time']    #total time interval
        IntervalTime = TotalTimeLine/TIME_STEPS       #devided equally 12 is timestep presetting
        


        PreConInt = []     #record Uk-1
        while True:
            output = []
            tweetIndex = 0
            if TotalTimeLine == 0:
                output.append(''.join(tweet["text"] for tweet in TweetList))
                break
            Start = TweetList[0]['time']
            Interval = int(TotalTimeLine/IntervalTime)
            Intset = []
            for inter in range(Interval):
                ifempty = 0
                interval = []
                sentence = ''
                for q in range(tweetIndex, len(TweetList)):
                    if TweetList[q]['time'] >= Start and TweetList[q]['time'] < Start+IntervalTime:
                        ifempty += 1
                        interval.append(TweetList[q]['text'])
                    elif TweetList[q]['time'] >= Start+IntervalTime:
                        tweetIndex = q-1   #record the last one 
                        break
                if ifempty == 0:
                    output.append([])
                else:
                    if TweetList[-1]['time'] == Start+IntervalTime:
                        interval.append(TweetList[-1]["text"])
                    Intset.append(inter)
                    sentence = ''.join(s for s in interval) 
                    output.append(sentence)
                    Start = Start+IntervalTime
            ConInt = ContinuousInterval(Intset)
            if len(ConInt)<TIME_STEPS and len(ConInt) > len(PreConInt):
                IntervalTime = int(IntervalTime*0.5)
                PreConInt = ConInt
                if IntervalTime == 0:
                    output = output[ConInt[0]:ConInt[-1]+1]
                    break
            else:
                output = output[ConInt[0]:ConInt[-1]+1]
                break
        """
               for q in range(len(output)):
            output[q] = ''.join(s for s in output[q]) 
        """
        
        
        totalData.append(output)
    return totalData
        
  

        

In [6]:
x_dev = preProcessData('dev.data.jsonl')
x_train = preProcessData('train.data.jsonl')
x_test = preProcessData('test.data.jsonl')

In [7]:
x_dev

[['#BREAKING Reports: 2 brothers suspected of Charlie Hebdo attack are dead http://t.co/7tGdSv4fr8',
  '“@USATODAY: #BREAKING Reports: 2 brothers suspected of Charlie Hebdo attack are dead http://t.co/R3DF0nzlBU”'],
 ['You are not alone today #Ottawa - we are here if you need to talk, cry, whatever, we are here 613-238-3311'],
 ['Have said it before, but needs saying again: Stop asking Muslims to condemn terrorism. It\'s bigoted and Islamophobic. http://t.co/6dzrywffN1@Max_Fisher @lpolgreen Its none of the above. Its a plea to get moderates to protest in muslim countries to end this savagery ruining Islam.In the same, stop bringing up slavery to white people who weren\'t slave owners because...wait, never mind because Narrative. @Max_Fisher@Max_Fisher How about asking everyone to condemn terrorism every timeBahahahaa RT @Max_Fisher: Have said it before. Stop asking Muslims to condemn terrorism. It\'s bigoted and Islamophobic.@Max_Fisher islamophobia is usually exaggerated and overblown